In [ ]:
import pandas as pd
import utils
import os
import ft_ensemble

In [ ]:
path_2_9km_models =  r'D:\SynologyDrive\DenseSM_9km' # path_2_pretrained_DenseSM-E
# source 50m
input_fine = pd.read_csv('E:\Demo\samples\input_fine.csv',index_col='s_index')
# source 9km
input_9km = pd.read_csv('E:\Demo\samples\input_9km.csv')

In [ ]:
setup = {'lr':5e-4, # learning rate
        'epoch_Num':50,# number of epochs
        'swa_start':40,
        'alpha':0.7,# alpha in eq. 3
        'beta':'auto',# beta in eq. 3 and was determined by eq.4
        'domain_type':'coral',
        'mv_type':'MAPE',
        'ex':'ft13_models',
        'batchS':128,# size of batch
        'br':1}# control the numbmer of unlabled samples and 9km samples, 1 means 1*batch_size
network_name=['SCAN']# used for training and testing in the transductive mode
base_dir = r'E:\Demo' #%network_name
size_r =list(range(1,22,4))+[29,37,45]# number of stations used for training
model_dir_e = os.path.join(base_dir,setup['ex'])
if not os.path.exists(model_dir_e):
    os.mkdir(model_dir_e)

In [ ]:
input_9km=utils.get_9km_of_a_network(input_fine,input_9km,network_name)
for nsample in size_r:
    model_dir = os.path.join(model_dir_e,'n_%s'%str(nsample))
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    for r in range(10):# 10 implementation
        if isinstance(setup['beta'],str):
            model_dir_r = os.path.join(model_dir,'a%s_b%s_r%s'%(int(setup['alpha']*100),setup['beta'],str(r)))
        else:
            model_dir_r = os.path.join(model_dir,'a%s_b%s_r%s'%(int(setup['alpha']*100),int(setup['beta']*100),str(r)))
        if not os.path.exists(model_dir_r):
            os.mkdir(model_dir_r)
        train_val_index = utils.get_train_val_index(input_fine,nsample,network=network_name)
        trainloader, val_data,targetloader,train_data,loader_9km = utils.prepare_train_val_data_multiscale(input_fine,train_val_index,input_9km,setup['batchS'],setup['br'])
        modelX=ft_ensemble.Build_DenseSM(path_2_9km_models)
        modelX.rebuild_DenseSME()
        data={'sl':trainloader,
             'tl':targetloader,
             'cl':loader_9km,
             'val_data':val_data,
             'train_data':train_data}
        print(f'Finetune sample:{nsample} repeat:{r}')
        ft=ft_ensemble.FinetuneModel(setup,data)
        ft.ft_ensemble(modelX,model_dir_r)
        df, res, m_specific_y=ft_ensemble.ensemble_results(model_dir_r, val_data)
        res.scatter_density_fig(os.path.join(model_dir_r,'ensemble.jpg'))
        print(res.stat_3d)